# Examen ETL: SPARK 12+1/02

Se podrá utilizar toda la información que se encuentra en el campus. 

Se va a trabajar sobre varios ficheros de datos:

Usuarios: id_usuario::sexo::edad::id_profesion::codigo_postal

Peliculas: id_pelicula::titulo (año)::tipo1|tipo2|tipo3

Ratings: id_pelicula::id_usuario::puntuacion::fecha_timestamp

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

In [66]:
# Nota: Carga de las puntuaciones
# Función para parsear la fecha

from datetime import datetime
dateparse = lambda x: datetime.fromtimestamp(float(x))

## Primera tarea: Inicializar spark context y cargar los datos desde los ficheros.

En primer lugar cargamos las librerías pertinentes e inicializamos Spark Context.

In [67]:
from pyspark.sql import Row
from pyspark import SparkContext
import numpy as np
import pandas as pd
sc= SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-af6f304ebf93>:5 

Se cargan los archivos y se comprueba que estan cargados correctamente.

In [68]:
user_ratings = sc.textFile("textfileexamen.txt")
movies = sc.textFile("movies.dat.txt")
ratings = sc.textFile("ratings.dat.txt")
users = sc.textFile("users.dat.txt")

In [69]:
user_ratings.take(1)

['1::F::1::10::48067::1193::5::978300760']

In [70]:
users.take(1)

['1::F::1::10::48067']

In [71]:
movies.take(1)

["1::Toy Story (1995)::Animation|Children's|Comedy"]

In [72]:
ratings.take(1)

['1::1193::5::978300760']

Comprobamos que están bien cargados y parseamos los datasets. Despues de parsearlos se comprueba que está bien hecho.

In [73]:
user_ratings_parseo = user_ratings.map(lambda x: x.split('::')).map(lambda x: Row(IDUsuario = int(x[0]), 
                                                                                  Sexo = x[1], 
                                                                                  Edad = int(x[2]), 
                                                                                  IDProf = x[3], 
                                                                                  CP = x[4], 
                                                                                  IDPelicula = x[5],
                                                                                 Puntuacion = int(x[6]),
                                                                                 Fecha = dateparse(x[7])))

In [74]:
user_ratings_parseo.take(1)

[Row(CP='48067', Edad=1, Fecha=datetime.datetime(2000, 12, 31, 22, 12, 40), IDPelicula='1193', IDProf='10', IDUsuario=1, Puntuacion=5, Sexo='F')]

In [75]:
users_parseo = users.map(lambda x: x.split('::')).map(lambda x: Row(IDUsuario = int(x[0]),
                                                                      Sexo = x[1],
                                                                    Edad = int(x[2]),
                                                                    IDProf = x[3],
                                                                    CP = x[4]))

In [76]:
users_parseo.take(1)

[Row(CP='48067', Edad=1, IDProf='10', IDUsuario=1, Sexo='F')]

In [77]:
movies_parseo = movies.map(lambda x: x.split('::')).map(lambda x: Row(IDPelicula = x[0],
                                                                     Year = x[1],
                                                                     Tipo = x[2]))

In [78]:
movies_parseo.take(1)

[Row(IDPelicula='1', Tipo="Animation|Children's|Comedy", Year='Toy Story (1995)')]

In [79]:
ratings_parseo = ratings.map(lambda x: x.split('::')).map(lambda x: Row(IDUsuario = int(x[0]),
                                                                          IDPelicula = int(x[1]),
                                                                          Puntuacion = int(x[2]),
                                                                          Fecha = dateparse(x[3])))

In [80]:
ratings_parseo.take(1)

[Row(Fecha=datetime.datetime(2000, 12, 31, 22, 12, 40), IDPelicula=1193, IDUsuario=1, Puntuacion=5)]

## Segunda tarea: Media de puntuaciones globales por año. ¿Hay algún año significativamente distinto?

In [81]:
puntuaciones_year = ratings_parseo.map(lambda x: (x.Fecha.year, x.Puntuacion))

aTuple = (0, 0)
puntuaciones_year.aggregateByKey(aTuple, lambda a,b: (a[0] + b,    a[1] + 1),\
                                 lambda a,b: (a[0] + b[0], a[1] + b[1]))\
                                .mapValues(lambda x: x[0]/x[1])\
                                .sortBy(lambda x: x[0], False)\
                                .collect()

[(2003, 3.486559139784946),
 (2002, 3.458828911253431),
 (2001, 3.5122542537247643),
 (2000, 3.5903916742285498)]

## Tercera pregunta: ¿Cuál es la película más votada por los mayores de 60? 

In [82]:
Mas_votos = (user_ratings_parseo.filter(lambda x: (x.Edad > 45))
              .map(lambda linea: (x.IDPelicula, x.IDUser))
              .groupByKey()
              .mapValues(set)
              .map(lambda x: (x[0], len(x[1]))))

In [ ]:
Mas_votos.take(1)

## Cuarta pregunta: ¿Cuál es la puntuación media de las peliculas de acción del año 2000?

## Quinta pregunta: ¿ Cuál es el año en que mayor número de usuarios votaron?

In [83]:
votaciones_year = (user_ratings_parseo.map(lambda x:(x.Fecha.year, x.IDUser))
       .groupByKey()
       .mapValues(set)
       .map(lambda x: (x[0], len(x[1]))))

In [84]:
votaciones_year.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 45, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 1497, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'IDUser' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, it

## Sexta pregunta: ¿ Cuál es la película con mejor puntación media?

In [ ]:
MediaFilms = user_ratings_parseo.map(lambda x: (x.IDPelicula, x.Puntuacion))

tuples = (0, 0)

rateFilms.aggregateByKey(tuples,  lambda a,b:(a[0]+b, a[1] + 1),\
                                 lambda a,b:(a[0]+b[0], a[1] + b[1]))\
                                .mapValues(lambda x: x[0]/x[1])\
                                .sortBy(lambda x: x[0], False)\
                                .collect()

In [ ]:
peliculas_media = puntPelis.takeOrdered(1, lambda x: -x[1])